In [66]:
import numpy as np
import pandas as pd
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
import string
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
import xgboost as xgb
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding,SpatialDropout1D
from tensorflow.keras.optimizers import RMSprop
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.callbacks import EarlyStopping,ModelCheckpoint
import keras
from keras.utils import pad_sequences
import plotly.express as px
import plotly.io as pio
import pickle

In [67]:
train1 = pd.read_csv("train.csv")
train1.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [68]:
train1_1 = train1.copy()
train1_1['label'] = train1_1['label'].astype(str)
train1_1.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [69]:
fig = px.histogram(train1_1, x = "label", color = "label")
fig.show()

In [70]:
train1.shape

(31962, 3)

In [71]:
train1.isnull().sum()

id       0
label    0
tweet    0
dtype: int64

In [72]:
train1.drop('id', axis = 1,inplace = True)
train1.head()

,label,tweet
0,0,@user when a father is dysfunctional and is s...
1,0,@user @user thanks for #lyft credit i can't us...
2,0,bihday your majesty
3,0,#model i love u take with u all the time in ...
4,0,factsguide: society now #motivation


In [73]:
train2 = pd.read_csv("labeled_data.csv")
train2.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [74]:
train2.shape

(24783, 7)

In [75]:
train2.isnull().sum()

Unnamed: 0            0
count                 0
hate_speech           0
offensive_language    0
neither               0
class                 0
tweet                 0
dtype: int64

In [76]:
train2.drop(['Unnamed: 0', 'count', 'hate_speech', 'offensive_language', 'neither'], axis = 1, inplace = True)
train2.head()

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [77]:
train2['class'].unique()

array([2, 1, 0], dtype=int64)

In [78]:
train2_1 = train2.copy()
train2_1['class'] = train2_1['class'].astype(str)
train2_1.head()

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [79]:
fig = px.histogram(train2_1, x = "class", color = "class")
fig.show()

In [80]:
# train2[train2['class'] == 0]['class'] = 1

In [81]:
train2.head()

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [82]:
train2['class'].unique()

array([2, 1, 0], dtype=int64)

In [83]:
train2[train2['class'] == 0]

,class,tweet
85,0,"""@Blackman38Tide: @WhaleLookyHere @HowdyDowdy1..."
89,0,"""@CB_Baby24: @white_thunduh alsarabsss"" hes a ..."
110,0,"""@DevilGrimz: @VigxRArts you're fucking gay, b..."
184,0,"""@MarkRoundtreeJr: LMFAOOOO I HATE BLACK PEOPL..."
202,0,"""@NoChillPaz: ""At least I'm not a nigger"" http..."
...,...,...
24576,0,this guy is the biggest faggot omfg
24685,0,which one of these names is more offensive kik...
24751,0,you a pussy ass nigga and I know it nigga.
24776,0,you're all niggers


In [84]:
train2["class"].replace({0: 1}, inplace = True)

In [85]:
train2['class'].unique()

array([2, 1], dtype=int64)

In [86]:
train2_1 = train2.copy()
train2_1['class'] = train2_1['class'].astype(str)
train2_1.head()

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [87]:
fig = px.histogram(train2_1, x = "class", color = "class")
fig.show()

In [88]:
train2[train2['class'] == 0]

,class,tweet


In [89]:
train2["class"].replace({2: 0}, inplace = True)
train2.head()

,class,tweet
0,0,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [90]:
train2_1 = train2.copy()
train2_1['class'] = train2_1['class'].astype(str)
train2_1.head()

,class,tweet
0,0,!!! RT @mayasolovely: As a woman you shouldn't...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [91]:
fig = px.histogram(train2_1, x = "class", color = "class")
fig.show()

In [92]:
train2.rename(columns = {'class':'label'}, inplace = True)

In [93]:
train2.iloc[0]['tweet']

"!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out..."

In [94]:
train2.iloc[5]['tweet']

'!!!!!!!!!!!!!!!!!!"@T_Madison_x: The shit just blows me..claim you so faithful and down for somebody but still fucking with hoes! &#128514;&#128514;&#128514;"'

In [95]:
data1 = [train1, train2]
data1 = pd.concat(data1)

In [96]:
data1.head()

,label,tweet
0,0,@user when a father is dysfunctional and is s...
1,0,@user @user thanks for #lyft credit i can't us...
2,0,bihday your majesty
3,0,#model i love u take with u all the time in ...
4,0,factsguide: society now #motivation


In [97]:
data1_1 = data1.copy()
data1_1['label'] = data1_1['label'].astype(str)
data1_1.head()

,label,tweet
0,0,@user when a father is dysfunctional and is s...
1,0,@user @user thanks for #lyft credit i can't us...
2,0,bihday your majesty
3,0,#model i love u take with u all the time in ...
4,0,factsguide: society now #motivation


In [98]:
fig = px.histogram(data1_1, x = "label", color = "label")
fig.show()

In [99]:
data1.shape

(56745, 2)

In [100]:
nltk.download('stopwords')
stemmer = nltk.SnowballStemmer("english")
stopword = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\vamsi\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [36]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = [word for word in text.split(' ') if word not in stopwords]
    text = " ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text = " ".join(text)
    return text

In [37]:
data1['tweet'] = data1['tweet'].apply(clean_text)

In [38]:
data1.head()

,label,tweet
0,0,user father dysfunct selfish drag kid dysfunc...
1,0,user user thank lyft credit cant use caus dont...
2,0,bihday majesti
3,0,model love u take u time urð± ððð...
4,0,factsguid societi motiv


In [39]:
X = data1['tweet']
y = data1['label']

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)
print(len(X_train), len(y_train))
print(len(X_test), len(y_test))

42558 42558
14187 14187


In [41]:
count = CountVectorizer(stop_words = 'english', ngram_range = (1,5))
X_train_vectorizer = count.fit_transform(X_train)
X_test_vectorizer = count.transform(X_test)

In [42]:
count.vocabulary_

{'pictur': 496965,
 'galleri': 244789,
 'live': 386374,
 'blackcavali': 78353,
 'caucasian': 105699,
 'webcam': 750931,
 'pictur galleri': 497099,
 'galleri live': 244792,
 'live blackcavali': 386462,
 'blackcavali caucasian': 78354,
 'caucasian webcam': 105705,
 'pictur galleri live': 497100,
 'galleri live blackcavali': 244793,
 'live blackcavali caucasian': 386463,
 'blackcavali caucasian webcam': 78355,
 'pictur galleri live blackcavali': 497101,
 'galleri live blackcavali caucasian': 244794,
 'live blackcavali caucasian webcam': 386464,
 'pictur galleri live blackcavali caucasian': 497102,
 'galleri live blackcavali caucasian webcam': 244795,
 'shoutout': 600018,
 'everybodi': 198100,
 'follow': 226779,
 'nicca': 455780,
 'shoutout everybodi': 600045,
 'everybodi follow': 198226,
 'follow nicca': 227264,
 'shoutout everybodi follow': 600046,
 'everybodi follow nicca': 198227,
 'shoutout everybodi follow nicca': 600047,
 'user': 703467,
 'bet': 56640,
 'second': 586131,
 'goal': 25

In [43]:
tfidf = TfidfTransformer()
X_train_tfidf = tfidf.fit_transform(X_train_vectorizer)
X_test_tfidf = tfidf.transform(X_test_vectorizer)

In [44]:
model_vectorizer = MultinomialNB().fit(X_train_vectorizer, y_train)
prediction_vectorizer = model_vectorizer.predict(X_test_vectorizer)
print(confusion_matrix(y_test,prediction_vectorizer))
print(classification_report(y_test, prediction_vectorizer))

[[7878  575]
 [ 458 5276]]
              precision    recall  f1-score   support

           0       0.95      0.93      0.94      8453
           1       0.90      0.92      0.91      5734

    accuracy                           0.93     14187
   macro avg       0.92      0.93      0.92     14187
weighted avg       0.93      0.93      0.93     14187



In [45]:
model_tfidf = MultinomialNB().fit(X_train_tfidf, y_train)
prediction_tfidf = model_tfidf.predict(X_test_tfidf)
print(confusion_matrix(y_test,prediction_tfidf))
print(classification_report(y_test, prediction_tfidf))

[[8213  240]
 [ 860 4874]]
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      8453
           1       0.95      0.85      0.90      5734

    accuracy                           0.92     14187
   macro avg       0.93      0.91      0.92     14187
weighted avg       0.92      0.92      0.92     14187



In [46]:
import xgboost as xgb
xgb_model = xgb.XGBClassifier(
        learning_rate = 0.1,
        max_depth = 7,
        n_estimators = 80,
        use_label_encoder = False,
        eval_metric ='auc' )

C:\Users\vamsi\anaconda3\lib\site-packages\xgboost\sklearn.py:1395: UserWarning:

`use_label_encoder` is deprecated in 1.7.0.



In [47]:
xgb_model_vectorizer = xgb_model.fit(X_train_vectorizer, y_train)
xgb_predictions_vectorizer = xgb_model_vectorizer.predict(X_test_vectorizer)
print(confusion_matrix(y_test,xgb_predictions_vectorizer))
print(classification_report(y_test, xgb_predictions_vectorizer))

[[8367   86]
 [ 923 4811]]
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      8453
           1       0.98      0.84      0.91      5734

    accuracy                           0.93     14187
   macro avg       0.94      0.91      0.92     14187
weighted avg       0.93      0.93      0.93     14187



In [48]:
xgb_model = xgb_model.fit(X_train_tfidf, y_train)
xgb_predictions = xgb_model.predict(X_test_tfidf)
print(confusion_matrix(y_test,xgb_predictions))
print(classification_report(y_test, xgb_predictions))

[[8358   95]
 [ 939 4795]]
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      8453
           1       0.98      0.84      0.90      5734

    accuracy                           0.93     14187
   macro avg       0.94      0.91      0.92     14187
weighted avg       0.93      0.93      0.93     14187



In [49]:
max_words = 50000
max_len = 300
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
sequences = tokenizer.texts_to_sequences(X_train)
sequences_matrix = keras.utils.pad_sequences(sequences,maxlen = max_len)

In [50]:
model = Sequential()
model.add(Embedding(max_words, 100, input_length = max_len))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout = 0.2, recurrent_dropout = 0.2))
model.add(Dense(1, activation = 'sigmoid'))
model.summary()
model.compile(loss = 'binary_crossentropy', optimizer = RMSprop(), metrics = ['accuracy'])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 100)          5000000   
                                                                 
 spatial_dropout1d (SpatialD  (None, 300, 100)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 5,080,501
Trainable params: 5,080,501
Non-trainable params: 0
_________________________________________________________________


In [51]:
stop = EarlyStopping(
    monitor = 'val_accuracy', 
    mode = 'max',
    patience = 5
)

checkpoint = ModelCheckpoint(
    filepath ='./',
    save_weights_only = True,
    monitor ='val_accuracy',
    mode ='max',
    save_best_only = True)

In [52]:
history = model.fit(sequences_matrix, y_train, batch_size = 1024, epochs = 15,
          validation_split = 0.2, callbacks=[stop,checkpoint])

Epoch 1/15
34/34 [==============================] - 432s 13s/step - loss: 0.4913 - accuracy: 0.7847 - val_loss: 0.3154 - val_accuracy: 0.9025
Epoch 2/15
34/34 [==============================] - 412s 12s/step - loss: 0.2371 - accuracy: 0.9242 - val_loss: 0.2042 - val_accuracy: 0.9301
Epoch 3/15
34/34 [==============================] - 419s 12s/step - loss: 0.1490 - accuracy: 0.9494 - val_loss: 0.1633 - val_accuracy: 0.9417
Epoch 4/15
34/34 [==============================] - 426s 12s/step - loss: 0.1109 - accuracy: 0.9612 - val_loss: 0.1623 - val_accuracy: 0.9410
Epoch 5/15
34/34 [==============================] - 416s 12s/step - loss: 0.0929 - accuracy: 0.9680 - val_loss: 0.1572 - val_accuracy: 0.9443
Epoch 6/15
34/34 [==============================] - 467s 14s/step - loss: 0.0769 - accuracy: 0.9739 - val_loss: 0.1613 - val_accuracy: 0.9429
Epoch 7/15
34/34 [==============================] - 438s 13s/step - loss: 0.0688 - accuracy: 0.9773 - val_loss: 0.1834 - val_accuracy: 0.9421
Epoch 

In [55]:
test_sequences = tokenizer.texts_to_sequences(X_test)
test_sequences_matrix = keras.utils.pad_sequences(test_sequences, maxlen = max_len)

In [56]:
accr = model.evaluate(test_sequences_matrix, y_test)

444/444 [==============================] - 33s 75ms/step - loss: 0.2406 - accuracy: 0.9266


In [57]:
lstm_prediction = model.predict(test_sequences_matrix)

444/444 [==============================] - 33s 74ms/step


In [58]:
res = []
for prediction in lstm_prediction:
    if prediction[0] < 0.5:
        res.append(0)
    else:
        res.append(1)

In [59]:
print(confusion_matrix(y_test, res))

[[7877  576]
 [ 466 5268]]


In [60]:
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol = pickle.HIGHEST_PROTOCOL)

In [61]:
model.save("hate&abusive_model.h5")

In [62]:
load_model = keras.models.load_model("./hate&abusive_model.h5")
with open('tokenizer.pickle', 'rb') as handle:
    load_tokenizer = pickle.load(handle)

In [104]:
test = 'Zelensky is a bitch ass clown. All he does is warmongering with the confidence that NATO will save him and his country\'s ass!!!' 

def clean_text(text):
    print(text)
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    print(text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text
test = [clean_text(test)]
seq = load_tokenizer.texts_to_sequences(test)
padded = keras.utils.pad_sequences(seq, maxlen=300)
pred = load_model.predict(padded)
print("pred", pred)
if pred < 0.5:
    print("no hate")
else:
    print("hate and offensive")

Zelensky is a bitch ass clown. All he does is warmongering with the confidence that NATO will save him and his country's ass!!!
zelensky is a bitch ass clown all he does is warmongering with the confidence that nato will save him and his countrys ass
1/1 [==============================] - 0s 87ms/step
pred [[0.9970021]]
hate and offensive


In [103]:
test1 = 'There\'s a lot of hatred being spread over Elon Musk which is not fair. Man\'s really a spectacular innovator who aspires to contribute towards humanity.'

def clean_text(text):
    print(text)
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    print(text)
    text = [word for word in text.split(' ') if word not in stopword]
    text=" ".join(text)
    text = [stemmer.stem(word) for word in text.split(' ')]
    text=" ".join(text)
    return text
test1 = [clean_text(test1)]
seq = load_tokenizer.texts_to_sequences(test1)
padded = keras.utils.pad_sequences(seq, maxlen=300)
pred = load_model.predict(padded)
print("pred", pred)
if pred < 0.5:
    print("no hate")
else:
    print("hate and offensive")

There's a lot of hatred being spread over Elon Musk which is not fair. Man's really a spectacular innovator who aspires to contribute towards humanity.
theres a lot of hatred being spread over elon musk which is not fair mans really a spectacular innovator who aspires to contribute towards humanity
1/1 [==============================] - 0s 85ms/step
pred [[0.00760976]]
no hate
